# Introduction

Example script for downloading SPARTACUS data.

**Website only allows for 6 months to 1 year's worth of data to be downloaded at a time, depending on the size of the lat-lon box.**

# Setup

## Modules

In [1]:
import datetime
from pathlib import Path

from ZAMGdatahub import data_download, query, utils

## Global variables

In [2]:
ODIR = "/home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/air_temp/grids/SPARTACUS"
# ODIR = "/home/skalevag/Documents/SPARTACUS_Tirol"

ODIR = Path(ODIR)

if not ODIR.is_dir():
    ODIR.mkdir(parents=True)

# variable
params = "Tn"  # minimum air temperature 2m above ground
# params = "Tx" # maximum air temperature 2m above ground
# params = "RR" # precipitation sum

maxMonths = 12

overwrite = False
overwriteMerge = False
verbose = True

startYear = 1961  # inclusive
endYear = 2021  # inclusive

datetimeformat = "%Y-%m-%d %H:%M"

In [3]:
# gridbox for Ötztal area
gridbox_oetz = query.LatLonBox(
    label="oetztal", lat_min=46.6, lat_max=47.3, lon_min=10.5, lon_max=11.4
)
print(gridbox_oetz)
print()

# gridbox for all Tirol
gridbox_tirol = query.LatLonBox(
    label="tirol", lat_min=46.77, lat_max=47.71, lon_min=9.53, lon_max=13.51
)
print(gridbox_tirol)
print()

# point location
loc_obergurgl = query.LatLonLocation("obergurgl", 46.8703, 11.0269)
print(loc_obergurgl)
print()

# dataset type
dataset = query.DatasetType.SPARTACUS
print(dataset)

LOCATION: oetztal
COORDINATES: latitude: 46.6 ... 47.3 ; longitude 10.5 ... 11.4

LOCATION: tirol
COORDINATES: latitude: 46.77 ... 47.71 ; longitude 9.53 ... 13.51

LOCATION: obergurgl
COORDINATES: latitude: 46.8703 ; longitude 11.0269

DatasetType.SPARTACUS


## Make query

In [4]:
myQuery = query.rasterQuery(dataset, params, gridbox=gridbox_oetz)

print(myQuery)

ZAMGdatahubQuery for download of SPARTACUS with parameters Tn. 
params: ['Tn']
dataset: DatasetType.SPARTACUS
location_label: oetztal
output_filename_head: spartacus-daily
output_format: netcdf
lat_min: 46.6
lat_max: 47.3
lon_min: 10.5
lon_max: 11.4


## Save/Load query to/from file

In [5]:
myQuery.saveQuery()

Query saved to "./SPARTACUS_query_oetztal.txt"


In [6]:
print(query.loadQuery("./SPARTACUS_query_oetztal.txt"))

ZAMGdatahubQuery for download of SPARTACUS with parameters Tn. 
params: ['Tn']
dataset: DatasetType.SPARTACUS
location_label: oetztal
output_filename_head: spartacus-daily
output_format: netcdf
lat_min: 46.6
lat_max: 47.3
lon_min: 10.5
lon_max: 11.4


# Download gridbox

In [7]:
# get all in a year
for year in range(startYear, endYear + 1):
    slices = utils.makeTimeSlices(
        year, maxMonths=maxMonths, firstDOY=datetime.datetime(1961, 1, 1, 0, 0)
    )
    for start, end in slices:
        data_download.downloadData(
            myQuery, start, end, ODIR, overwrite=overwrite, verbose=verbose
        )

spartacus-daily_Tn_oetztal_196101010000-196201010000.nc has already been downloaded: /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/air_temp/grids/SPARTACUS/spartacus-daily_Tn_oetztal_196101010000-196201010000.nc
spartacus-daily_Tn_oetztal_196201010000-196301010000.nc has already been downloaded: /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/air_temp/grids/SPARTACUS/spartacus-daily_Tn_oetztal_196201010000-196301010000.nc
spartacus-daily_Tn_oetztal_196301010000-196401010000.nc has already been downloaded: /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/air_temp/grids/SPARTACUS/spartacus-daily_Tn_oetztal_196301010000-196401010000.nc
spartacus-daily_Tn_oetztal_196401010000-196501010000.nc has already been downloaded: /home/skalevag/Documents/NRC_P8_water_energy_and_sediment/data/air_temp/grids/SPARTACUS/spartacus-daily_Tn_oetztal_196401010000-196501010000.nc
spartacus-daily_Tn_oetztal_196501010000-196601010000.nc has already been downloaded:

## Merge NetCDF files by year
Not really needed for the SPARTACUS data.

In [8]:
if maxMonths != 12:
    for year in range(startYear, endYear + 1):
        data_download.mergeNetCDFfilesByYear(
            year, ODIR, overwrite=overwriteMerge, verbose=verbose
        )

# Download point location

In [9]:
# select all the available INCA parameters
params = ["Tn", "Tx", "RR"]
myQuery = query.rasterQuery(
    query.DatasetType.SPARTACUS_POINT, params, point_location=loc_obergurgl
)

print(myQuery)

ZAMGdatahubQuery for download of SPARTACUS_POINT with parameters Tn,Tx,RR. 
params: ['Tn', 'Tx', 'RR']
dataset: DatasetType.SPARTACUS_POINT
location_label: obergurgl
output_filename_head: spartacus-point
output_format: csv
lat: 46.8703
lon: 11.0269


In [10]:
start = "1961-01-01 00:00"
end = "2022-05-01 00:00"
url = data_download.makeURL(myQuery, start, end)
print(url)

https://dataset.api.hub.zamg.ac.at/v1/timeseries/historical/spartacus-v1-1d-1km?anonymous=true&parameters=Tn,Tx,RR&start=1961-01-01T00:00&end=2022-05-01T00:00&lon=11.0269&lat=46.8703&output_format=csv
